# The Battle of Neighborhoods

# 2 - DATA

### Description of the problem and a discussion of the background:

#### I am Shah Nawaj Rahman,
A data enthusiast residing in Mirpur, Dhaka, Bangladesh.

The following data is required to  answer the issues of the problem:

- List of neighborhood in Banani with geo data (latitud and longitud) 
- List of  Bus stations in Banani with their address location 
- List of house for rent in Banani area with their addresses and price
- Preferably, a list of apartment for rent with additional information, such as price, address, area, room, etc
- Venues for each Banani neighborhood ( than can be clustered)


### How the data will be used to solve the problem

The data will be used as follows:
- Use Foursquare and geopy data to map top 10 venues for all Banani neighborhoods and clustered in groups ( as per Course LAB)
- Use foursquare and geopy data to map the location of bus stations , separately and on top of the above clustered map in order to be able to identify the venues and ammenities
- create a map that depicts, for instance, the average rental price, around a radious of 1.0 km around each bus station - or a similar metrics.
- Addresses from rental locations will be converted to geodata( lat, long) using Geopy-distance and Nominatim.
- Data will be searched in open data sources if available, from real estate sites if open to reading, and libraries.


The procesing of these DATA will allow to answer the key questions to make a decision:
- what is the cost of rent around a 1.0 km radius from each bus station?
- what is the area of Banani with best rental pricing that meets criteria established?
- What is the distance from work place (Banani Commercial Area) and the tentative future home?
- What are the venues of the two best places to live? How the prices compare?
- How venues distribute among Banani neighborhoods and around bus stations?
- Are there tradeoffs between size and price and location?
- Any other interesting statistical data findings of the real estate and overall data.

In [1]:
import numpy as np
import pandas as pd
import time
import json
import requests
import folium

from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print('Libraries imported!')

Libraries imported!


## 3 - STUDY OF MY CURRENT LOCATION

### Reference of venues around current residence in Mirpur for comparison to Banani

In [2]:
address = 'Mirpur, Dhaka'

geolocator = Nominatim(user_agent=str(address))
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mirpur home are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Mirpur home are 23.8122474, 90.3597632.


In [3]:
neighborhood_latitude=23.8122474
neighborhood_longitude=90.3597632

In [4]:
CLIENT_ID = 'BLTEZU0PVRN3UR2NOYPATUS4PKJ4S0XVRYIU5S0LETHFMZSO' # Put Your Client Id
CLIENT_SECRET = '5YDVXPY1ORR5IUM0TED4SLT1UGNZKIAQH2HQQFT5YXQNOEUM' # Put You Client Secret 
VERSION = '20210101'
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

print (url)
print ('url creation for Mirpur complete')

https://api.foursquare.com/v2/venues/explore?&client_id=BLTEZU0PVRN3UR2NOYPATUS4PKJ4S0XVRYIU5S0LETHFMZSO&client_secret=5YDVXPY1ORR5IUM0TED4SLT1UGNZKIAQH2HQQFT5YXQNOEUM&v=20210101&ll=23.8122474,90.3597632&radius=1000&limit=100
url creation for Mirpur complete


In [5]:
results = requests.get(url).json()
#results

In [6]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [7]:
venues = results['response']['groups'][0]['items']
    
MPnearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
MPnearby_venues = MPnearby_venues.loc[:, filtered_columns]

# filter the category for each row
MPnearby_venues['venue.categories'] = MPnearby_venues.apply(get_category_type, axis=1)

# clean columns
MPnearby_venues.columns = [col.split(".")[-1] for col in MPnearby_venues.columns]

MPnearby_venues.head()

C:\Users\Shah\AppData\Local\Temp/ipykernel_8680/3882358043.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  MPnearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Sher-e-Bangla National Cricket Stadium,Cricket Ground,23.806196,90.363579
1,Original 10 Mirpur,Other Great Outdoors,23.810282,90.366177
2,Rabbani Hotel and Restaurant,Restaurant,23.815782,90.366411
3,Selim's Special Tea,Tea Room,23.809275,90.364614
4,Xinxian Resturant,Chinese Restaurant,23.813651,90.366475


## Map of Mirpur with venues near residence place - for reference 

In [8]:
map_mp = folium.Map(location=[latitude, longitude], zoom_start=15)

# add markers to map
for lat, lng, label in zip(MPnearby_venues['lat'], MPnearby_venues['lng'], MPnearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=100,
        radius=10,
        popup=label,
        color='deeppink',
        fill_color='#0f0f0f',
        fill_opacity=0.6,
    ).add_to(map_mp)  
    
map_mp